# TITLE I

## Lib's

In [1]:
import pandas as pd
import re
import os

from dash import Dash
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash import dcc
from dash import html

import plotly.express as px
import base64


## Dataset

In [ ]:
#reading data
path = 'data'
lista = []
df = pd.DataFrame()

for f in os.listdir(path + "/"):
    frames = pd.read_csv(path + "/" + f, encoding = 'latin-1', sep = ';', low_memory = False)
    file = re.search(r'\d+', f).group(0)
    
    frames['file'] = file
    df = df.append(frames, ignore_index=True)

In [ ]:
df.head()

In [ ]:
# data type
df['data_inversa'] = df['data_inversa'].astype('datetime64')

print(df['data_inversa'].min())
print(df['data_inversa'].max())

In [ ]:
# add new columns
df['month'] = df['data_inversa'].dt.month
df['year'] = df['data_inversa'].dt.year

df['time'] = df['horario'].str.slice(0, 2)

print(df['horario'].min(), df['horario'].max())

In [ ]:
df.head(3).T

In [ ]:
# getting only the target columns
df1 = df[['dia_semana', 'uf', 'br', 'km',
       'municipio', 'causa_acidente', 'tipo_acidente',
        'fase_dia', 
       'condicao_metereologica', 'tipo_pista', 
       'pessoas', 'mortos', 'feridos_leves', 'feridos_graves', 'ilesos',
       'ignorados', 'feridos', 'file', 'month', 'year', 'time']]

In [ ]:
#replacing some values
df1 = df1.copy(deep = True)

df1['dia_semana'] = [x.lower() for x in df1['dia_semana']]
df1['dia_semana'] = [x.replace('-feira', '') for x in df1['dia_semana']]

df1['causa_acidente'] = df1['causa_acidente'].str.lower()

dict_replace = {'ingestão de álcool e/ou substâncias psicoativas pelo pedestre' : 'ingestão de álcool',
               'ingestão de álcool ou de substâncias psicoativas pelo pedestre' : 'ingestão de álcool',
               'ingestão de álcool pelo condutor' : 'ingestão de álcool',
               'redutor de velocidade em desacordo' : 'velocidade incompatível',
               'dormindo' : 'condutor dormindo',
               'falta de atenção à condução' : 'falta de atenção',
               'defeito mecânico em veículo' : 'defeito mecânico no veículo',
               'mal súbito do condutor' : 'mal súbito'}

df1 = df1.replace({'causa_acidente' : dict_replace})


In [ ]:
df1['dia_semana'].unique()

# PLOTLY DASHBOARD

In [ ]:
df1.columns

In [ ]:
df1['tipo_acidente'].unique()

In [ ]:
# app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY],
#                 meta_tags = [{'name' : 'viewport',
#                               'content' : 'width=dice_width, initial-scale=1.0'}])

# app.layout = dbc.Container([
#     html.P(' '),
#     html.Div(id = '1st-line-separator', style = {'border' : '4px LightCoral solid'}),   
#         dbc.Card([
#             dbc.CardBody([
#                 html.H1('ACIDENTES RODOVIÁRIOS', style = {'color' : '#F8F9F9'},
#                    className = 'text-center'),
#                 html.H5('2008 à 2021', style = {'color' : '#F8F9F9'},
#                    className = 'text-center')
#                     ],  style={'border' : 'none'})
#                 ]),
        
#     html.P(' '),
#     html.Div([
#         html.Span('Dashboard referente a acidentes rodoviários registrados no períiodo de 2008 à 2021\ne disponibilizados pela Polícia Federal do Brasil'),
#         ], style = {'color' : '#A6ACAF', 'fontSize': 14, 'font-weight': 'light'},
#     className = 'text-center'),
#     html.P(' '),    
#     html.Div(id = '2st-line-separator', style = {'border' : '1px Lightpink solid'}),
#     html.P(' '),
#     dbc.Card([
#     html.P(' '),  
#     dbc.Row([
#         dbc.Col([
#             html.H5("Anos", style = {'color' : 'DimGrey', 'font-size' : 15, 'font-weight':'normal' })
#         ], width = {'size' : 1,'offset' : 0.5}),
#         dbc.Col([
#             dcc.RangeSlider(id='year-slider',
#                             marks = {str(i) : i for i in df['year'].unique()},
#                             count = 1,
#                             min = df1['year'].min(),
#                             max = df1['year'].max(),
#                             step = 1,
#                             value = [2015, 2021],
#                             className="margin9")
#         ],  width = {'size' : 10,'offset' : 0}),
#         ]),
#     html.P(' '),
#     dbc.Row([
#         dbc.Col([     
#             html.H5('Meses: ', style = {'color' : 'DimGrey', 'font-size' : 15, 'font-weight':'normal' })
#         ], width = {'size' : 1,'offset' : 0.5}),
        
#     dbc.Col([
#         dcc.Dropdown(id='month-picker',
#                             multi = True, value = [1, 2],
#                             options = [{'label' : x, 'value' : x} for x in sorted(df1['month'].unique()) if x != '(null)'])
#     ], width = {'size' : 4,'offset' : 0}),
#     dbc.Col([
#             html.H5('Dia da semana: ', style = {'color' : 'DimGrey', 'font-size' : 15, 'font-weight':'normal' })
#     ], width = {'size' : 1, 'offset' : 0}),
#     dbc.Col([dcc.Dropdown(id = 'week-day-picker',
#                             multi = True, value = 'domingo',
#                             options = [{'label' : x, 'value' : x} for x in sorted(df1['dia_semana'].unique()) if x != '(null)'])
#     ], width = {'size' : 4, 'offset' : 0})
#         ]),
#     html.Div(id = '20st-line-separator', style = {'border' : '0px Gainsboro solid'}),
#     ]),
#     dbc.Card([
#     html.P(' '),
#     dbc.Row([
#         dbc.Col([
#         html.H5('Estados: ', style = {'color' : 'DimGrey', 'font-size' : 15, 'font-weight':'normal' })
#     ], width = {'size' : 1, 'offset' : 0}),    
#     dbc.Col([
#         dcc.Dropdown(id='state-picker',
#                             multi = True, value = [1, 2],
#                             options = [{'label' : x, 'value' : x} for x in sorted(df1['uf'].unique()) if x != '(null)'])
#     ], width = {'size' : 9,'offset' : 0}),
#         html.P(' ')
#     ])
#         ]),
#     dbc.Col([
#             dcc.Graph(id='line-plot', figure = {},
#             style = {'border' : '1px Azure Solid'})
#         ], xs = 12, sm = 12, md = 12, lg = 9, xl = 9)

#     ])


# @app.callback(
#     Output('line-plot', 'figure'),
#     Input('year-slider', 'value')
# )
# def update_first_plot(year_selected):
#     df01 = df1[['year', 'mortos']].groupby(['year'], as_index = False).sum()
#     df01 = df1[df['year'].isin(year_selected)]
#     fig1 = px.line(df01, x ='Date', y = 'mortos', color = 'Symbols')
             
    

# if __name__ == '__main__':
#     app.run_server()

In [ ]:
# app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY],
#                 meta_tags = [{'name' : 'viewport',
#                               'content' : 'width=dice_width, initial-scale=1.0'}])

# app.layout = dbc.Container([
#     html.P(' '),
#     html.Div(id = '1st-line-separator', style = {'border' : '4px LightCoral solid'}),   
#         dbc.Card([
#             dbc.CardBody([
#                 html.H1('ACIDENTES RODOVIÁRIOS', style = {'color' : '#F8F9F9'},
#                    className = 'text-center'),
#                 html.H5('2008 à 2021', style = {'color' : '#F8F9F9'},
#                    className = 'text-center')
#                     ],  style={'border' : 'none'})
#                 ]),     
#     html.P(' '),
#     html.Div([
#         html.Span('Dashboard referente a acidentes rodoviários registrados no períiodo de 2008 à 2021\ne disponibilizados pela Polícia Federal do Brasil'),
#         ], style = {'color' : '#A6ACAF', 'fontSize': 14, 'font-weight': 'light'},
#     className = 'text-center'),
#     html.P(' '),    
#     html.Div(id = '2st-line-separator', style = {'border' : '1px Lightpink solid'}),
#     html.P(' '),
    
#     dbc.Row([
#         dbc.Col([
#             html.H1('Ano da ocorrência', style = {'color' : '#A6ACAF', 'fontSize' : 10,
#                                                     'font-weight' : 'light',
#                                                     'backgroundColor' : 'orange'})
#         ], width = {'size' : 2, 'offset' : 0}),
#         dbc.Col([
            
#             html.H1('Ano da ocorrência', style = {'color' : '#A6ACAF', 'fontSize' : 10, 'font-weight' : 'light',
#                                              'backgroundColor' : 'red'})
#             ], width = {'size' : 2,'offset' : 3}),
#     ]),
#     dbc.Row([
#         dbc.Col([
#             dcc.Slider(id = 'year-slider', min = 2007, max = 2021, value = 2007, step = 1,
#                       marks = {i: i for i in range(2007, 2021)})
#         ])
#     ])
# ])

# if __name__ == '__main__':
#     app.run_server()

In [ ]:
# app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY],
#                 meta_tags = [{'name' : 'viewport',
#                               'content' : 'width=dice_width, initial-scale=1.0'}])


# app.layout = dbc.Container([
#     html.P(' '),
#     html.Div(id = '1st-line-separator', style = {'border' : '4px LightCoral solid'}),   
#         dbc.Card([
#             dbc.CardBody([
#                 html.H2('ACIDENTES RODOVIÁRIOS', style = {'color' : '#F8F9F9'},
#                    className = 'text-center'),
#                 html.H5('2008 à 2021', style = {'color' : '#F8F9F9'},
#                    className = 'text-center')
#                     ],  style={'border' : 'none'})
#                 ]),     
#     html.P(' '),
#     html.Div([
#         html.Span('Dashboard referente a acidentes rodoviários registrados no período de 2008 à 2021\ne disponibilizados pela Polícia Federal do Brasil'),
#         ], style = {'color' : '#A6ACAF', 'fontSize': 10, 'font-weight': 'light'},
#     className = 'text-center'),
#     html.P(' '),    
#     html.Div(id = '2st-line-separator', style = {'border' : '2px #343a40 solid'}),
#     html.P(' '),
    
#     html.Div([
#     dbc.Row([
#         dbc.Col([
#             html.H1('Ano da ocorrência', style = {'color' : 'white', 'fontSize' : 9,
#                                                     'font-weight' : 'light',
#                                                   'font-weight' : 'light'
#                                                     }),
#             ],width = {'size' : 6}, className = 'col-ano-ocorrencia'),
        
#         dbc.Col([
#             html.H1('UF da ocorrência', style = {'color' : 'white', 'fontSize' : 9,
#                                             'font-weight' : 'light',
#                                             'font-weight' : 'light'}),
#         ], width = {'size' : 5}, className = 'col-uf-ocorrencia')
#     ], style = {'width' : '100%'}),
        
#     dbc.Row([
#         dbc.Col([
#             dcc.Slider(min = 2007, max = 2021, value = 2007, step = 1,
#                        className = 'year-slider',
#                       marks = {str(i) : i for i in df['year'].unique()})
#         ], width = {'size' : 6}),
        
#         dbc.Col([
#             dcc.Dropdown(options = ['SP', 'RJ', 'SE'],
#                          style={'height': '20px', 
#                                 'width' : '95%',
#                                 'display': 'inline-block'},
#                         className = 'dropdown-states'),
#         ], width = {'size' : 5}),
         

#         ], className = 'container-year-slider',
#             style = {'width' : '100%'})
#     ])
        
    
# ])
        

# if __name__ == '__main__':
#     app.run_server()

In [2]:
df = pd.read_csv('cars_accident_formated.csv', sep = ';', low_memory=False)

In [60]:
# app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY],
#                 meta_tags = [{'name' : 'viewport',
#                               'content' : 'width=dice_width, initial-scale=1.0'}])


# app.layout = dbc.Container([
#     html.P(' '),
#     html.Div(id = '1st-line-separator', style = {'border' : '4px LightCoral solid'}),   
#         dbc.Card([
#             dbc.CardBody([
#                 html.H2('ACIDENTES RODOVIÁRIOS', style = {'color' : '#F8F9F9'},
#                    className = 'text-center'),
#                 html.H5('2008 à 2021', style = {'color' : '#F8F9F9'},
#                    className = 'text-center')
#                     ],  style={'border' : 'none'})
#                 ]),     
#     html.P(' '),
#     html.Div([
#         html.Span('Dashboard referente a acidentes rodoviários registrados no período de 2008 à 2021\ne disponibilizados pela Polícia Federal do Brasil'),
#         ], style = {'color' : '#A6ACAF', 'fontSize': 10, 'font-weight': 'light'},
#     className = 'text-center'),
#     html.P(' '),    
#     html.Div(id = '2st-line-separator', style = {'border' : '2px #343a40 solid'}),
#     html.P(' '),
      
#     dbc.Row([
#         dbc.Col([
#             html.H1('Ano da ocorrência', style = {'color' : 'white', 'fontSize' : 9,
#                                                     'font-weight' : 'light',
#                                                   'font-weight' : 'light'}),
#         dcc.RangeSlider(id='selected_year_slider',
#                             marks = {str(i) : i for i in sorted(df['year'].unique())},
#                             count = 1,
#                              min = 2007,
#                              max = 2021,
#                              step = 1,
#                              className='year-slider'),
            
            
#             dcc.Graph(id='line-plot', figure = {}),
#             ],width = {'size' : 6}, className = 'col-ano-ocorrencia'),
        
            
        
#         dbc.Col([
#             html.H1('UF da ocorrência', style = {'color' : 'white', 'fontSize' : 9,
#                                             'font-weight' : 'light',
#                                             'font-weight' : 'light'}),
#             dcc.Dropdown(id='dropdown-uf',
#                          multi = True,
#                          options = [{'label' : x, 'value' : x} for x in sorted(df['uf'].unique()) if x != '(null)'],
#                          style={'height': '20px', 
#                                 'width' : '95%',
#                                 'display': 'inline-block'},
#                         className = 'dropdown-states'),
            
#             dcc.Graph(id = 'bar-plot-uf', figure = {})
            
#         ], width = {'size' : 6}, className = 'col-uf-ocorrencia')
#     ], style = {'width' : '100%'}, className = 'container-row-filters')

        
# ])


# @app.callback(
#     Output('line-plot', 'figure'),
#     Input('selected_year_slider', 'value'))
# def update_line_plot(selected_year_slider):
#     plot1 = df['year'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'year',
#             'year' : 'qt_acidentes'}).sort_values(by='year')
#     plot1 = plot1[plot1['year'].isin(selected_year_slider)]
#     fig1 = px.line(plot1, x = 'year', y = 'qt_acidentes')

#     fig1.update_layout(
#     {'plot_bgcolor' : 'rgba(0,0,0,0.0)',
#     'paper_bgcolor': 'rgba(0, 0, 0, 0)'}  )
#     fig1.update_xaxes(showgrid = False, showline= True)
#     fig1.update_yaxes(showgrid = False, showticklabels = True, title = None, showline= True)
     
#     return fig1
    
# @app.callback(
#     Output('bar-plot-uf', 'figure'),
#     Input('dropdown-uf', 'value'))
# def update_bar_plot(selected_uf):
#     plot2 = df['uf'].value_counts().to_frame().reset_index().rename(columns=({'index' : 'uf',
#                                                                  'uf' : 'qt'}))
#     #plot2 = plot2[:-1]
#     plot2 = plot2[plot2['uf'].isin(selected_uf)]
#     fig2 = px.bar(plot2, x = 'uf', y='qt')
    
#     fig2.update_layout(
#     {'plot_bgcolor' : 'rgba(0,0,0,0.0)',
#     'paper_bgcolor': 'rgba(0, 0, 0, 0)'}  )
#     fig2.update_xaxes(showgrid = False, showline= True)
#     fig2.update_yaxes(showgrid = False, showticklabels = True, title = None, showline= True)
#     return fig2
        
# #fig.update_layout(yaxis={'visible': False, 'showticklabels': False})


# if __name__ == '__main__':
#     app.run_server()

In [6]:
# px.line(df['year'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'year',
#             'year' : 'qt_acidentes'}).sort_values(by='year'), x = 'year', y = 'qt_acidentes')

In [ ]:
app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY],
                meta_tags = [{'name' : 'viewport',
                              'content' : 'width=dice_width, initial-scale=1.0'}])


app.layout = dbc.Container([
    html.P(' '),
    html.Div(id = '1st-line-separator', style = {'border' : '4px LightCoral solid'}),   
        dbc.Card([
            dbc.CardBody([
                html.H2('ACIDENTES RODOVIÁRIOS', style = {'color' : '#F8F9F9'},
                   className = 'text-center'),
                html.H5('2008 à 2021', style = {'color' : '#F8F9F9'},
                   className = 'text-center')
                    ],  style={'border' : 'none'})
                ]),     
    html.P(' '),
    html.Div([
        html.Span('Dashboard referente a acidentes rodoviários registrados no período de 2008 à 2021\ne disponibilizados pela Polícia Federal do Brasil'),
        ], style = {'color' : '#A6ACAF', 'fontSize': 10, 'font-weight': 'light'},
    className = 'text-center'),
    html.P(' '),    
    html.Div(id = '2st-line-separator', style = {'border' : '2px #343a40 solid'}),
    html.P(' '),
      
    html.Div([
    dbc.Row([
        dbc.Col([
            html.H1('Ano da ocorrência', style = {'color' : 'white', 'fontSize' : 9,
                                                    'font-weight' : 'light',
                                                  'font-weight' : 'light'}),
        dcc.RangeSlider(id='selected_year_slider',
                            marks = {str(i) : i for i in sorted(df['year'].unique())},
                            count = 1,
                             min = 2007,
                             max = 2021,
                             step = 1,
                             className='year-slider')
            ],width = {'size' : 6}, className = 'col-ano-ocorrencia'),
        dbc.Col([
            html.H1('UF da ocorrência', style = {'color' : 'white', 'fontSize' : 9,
                                            'font-weight' : 'light',
                                            'font-weight' : 'light'}),
            dcc.Dropdown(id='dropdown-uf',
                         multi = True,
                         options = [{'label' : x, 'value' : x} for x in sorted(df['uf'].unique()) if x != '(null)'],
                         style={'height': '20px', 
                                'width' : '95%',
                                'display': 'inline-block'},
                        className = 'dropdown-states'),
            ],width = {'size' : 6}, className = 'col-ano-ocorrencia')
        
    ], style = {'width' : '100%'}, className = 'container-row-filters'),
    
        dbc.Row([
            dcc.Graph(id='line-plot', figure = {})
        ])
    
])
        
])
# 


@app.callback(
    Output('line-plot', 'figure'),
    Input('selected_year_slider', 'value'))
def update_line_plot(selected_year_slider):
    plot1 = df['year'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'year',
            'year' : 'qt_acidentes'}).sort_values(by='year')
    plot1 = plot1[plot1['year'].isin(selected_year_slider)]
    fig1 = px.line(plot1, x = 'year', y = 'qt_acidentes')

    fig1.update_layout(
    {'plot_bgcolor' : 'rgba(0,0,0,0.0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)'}  )
    fig1.update_xaxes(showgrid = False, showline= True)
    fig1.update_yaxes(showgrid = False, showticklabels = True, title = None, showline= True)
     
    return fig1
    
# @app.callback(
#     Output('bar-plot-uf', 'figure'),
#     Input('dropdown-uf', 'value'))
# def update_bar_plot(selected_uf):
#     plot2 = df['uf'].value_counts().to_frame().reset_index().rename(columns=({'index' : 'uf',
#                                                                  'uf' : 'qt'}))
#     #plot2 = plot2[:-1]
#     plot2 = plot2[plot2['uf'].isin(selected_uf)]
#     fig2 = px.bar(plot2, x = 'uf', y='qt')
    
#     fig2.update_layout(
#     {'plot_bgcolor' : 'rgba(0,0,0,0.0)',
#     'paper_bgcolor': 'rgba(0, 0, 0, 0)'}  )
#     fig2.update_xaxes(showgrid = False, showline= True)
#     fig2.update_yaxes(showgrid = False, showticklabels = True, title = None, showline= True)
#     return fig2
        
# #fig.update_layout(yaxis={'visible': False, 'showticklabels': False})


if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

INFO:__main__:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/May/2022 10:27:43] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2022 10:27:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 10:27:43] "GET /assets/01-style.css?m=1652100980.7753317 HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [09/May/2022 10:27:43] "GET /assets/01-style.css?m=1652100980.7753317 HTTP/1.1" 304 -
127.0.0.1 - - [09/May/2022 10:27:44] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2022 10:27:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 10:27:44] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2022 10:27:44] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 10:27:44] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2022 10:27:44] "GET /_dash-component-suites/das

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/Users/daianeklein/minico

...

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/Users/daianeklein/minico

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/Users/daianeklein/minico

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/Users/daianeklein/minico

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/Users/daianeklein/minico

ERROR:__main__:Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/Users/daianeklein/miniconda3/envs/venv-dash/lib/python3.9/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/Users/dai